In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment')

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

### Data Loading

In [3]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df = df.copy()

    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda d: d.total_seconds()/60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    cat_vars = ['PULocationID','DOLocationID']
    df[cat_vars] = df[cat_vars].astype(str)

    return df

In [4]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [5]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
num_vars = ['trip_distance']
cat_vars = ['PU_DO']#'PULocationID','DOLocationID']

### Training set
- DictVectorizer maps all the values into a sparse matrix (for string variables, it performs one-hot encoding)

In [7]:
dv = DictVectorizer()

train_dicts = df_train[cat_vars + num_vars].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_train[target].values

### Validation set

In [8]:
val_dicts = df_val[cat_vars + num_vars].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_val = df_val[target].values

### Model 2 

In [14]:
ridge = Ridge(alpha=alpha)
ridge.fit(X_train,y_train)
y_pred = ridge.predict(X_val)

In [16]:
with open('./models/lin_reg.bin','wb') as f_out:
    pickle.dump((dv,ridge),f_out)

### Running different parameters for Model 2, logging with Mlflow and saving a model pickle as artifact
- log_param: enables logging every model parameter we want regarding our run
- log_metric: defines which metric we are interested in

In [17]:
with mlflow.start_run():
    
    mlflow.set_tag('developer','pedro')
    
    mlflow.log_param('train-data-path','./data/green_tripdata_2021-01.parquet')
    mlflow.log_param('validation-data-path','./data/green_tripdata_2021-02.parquet')
    
    alpha = 0.01
    
    mlflow.log_param('alpha',alpha)
    
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train,y_train)
    y_pred = ridge.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse',rmse)
    
    mlflow.log_artifact(local_path='models/lin_reg.bin',artifact_path='models_pickle')

### Example of trying new model (XGBoost) and performing hyperparameter tuning
- http://hyperopt.github.io/hyperopt/getting-started/search_spaces/

In [9]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [10]:
train = xgb.DMatrix(X_train,y_train)
valid = xgb.DMatrix(X_val,y_val)

In [25]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model','xgboost')
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid,'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric('rmse',rmse)
    
    return {'loss':rmse, 'status':STATUS_OK}

In [26]:
search_space = {
    'max_depth':scope.int(hp.quniform('max_depth',4,100,1)),
    'learning_rate':hp.loguniform('learning_rate',-3,0),
    'reg_alpha':hp.loguniform('reg_alpha',-5,1),
    'reg_lambda':hp.loguniform('reg_lambda',-6,-1),
    'min_child_weight':hp.loguniform('min_child_weight',-1,3),
    'objective':'reg:linear',
    'seed':42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[08:52:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:18.51207                                                                                           
[1]	validation-rmse:16.27793                                                                                           
[2]	validation-rmse:14.42391                                                                                           
[3]	validation-rmse:12.89638                                                                                           
[4]	validation-rmse:11.64431                                                                                           
[5]	validation-rmse:10.62571                                                                                           
[6]	validation-rmse:9.80366                                                                            

[134]	validation-rmse:6.51332                                                                                          
[135]	validation-rmse:6.51323                                                                                          
[136]	validation-rmse:6.51334                                                                                          
[137]	validation-rmse:6.51311                                                                                          
[138]	validation-rmse:6.51314                                                                                          
[139]	validation-rmse:6.51272                                                                                          
[140]	validation-rmse:6.51251                                                                                          
[141]	validation-rmse:6.51252                                                                                          
[142]	validation-rmse:6.51271           

[270]	validation-rmse:6.50893                                                                                          
[271]	validation-rmse:6.50889                                                                                          
[272]	validation-rmse:6.50862                                                                                          
[273]	validation-rmse:6.50865                                                                                          
[274]	validation-rmse:6.50847                                                                                          
[275]	validation-rmse:6.50853                                                                                          
[276]	validation-rmse:6.50854                                                                                          
[277]	validation-rmse:6.50860                                                                                          
[278]	validation-rmse:6.50876           

[113]	validation-rmse:6.53548                                                                                          
[114]	validation-rmse:6.53459                                                                                          
[115]	validation-rmse:6.53387                                                                                          
[116]	validation-rmse:6.53340                                                                                          
[117]	validation-rmse:6.53281                                                                                          
[118]	validation-rmse:6.53241                                                                                          
[119]	validation-rmse:6.53186                                                                                          
[120]	validation-rmse:6.53133                                                                                          
[121]	validation-rmse:6.53125           

[249]	validation-rmse:6.49611                                                                                          
[250]	validation-rmse:6.49591                                                                                          
[251]	validation-rmse:6.49585                                                                                          
[252]	validation-rmse:6.49579                                                                                          
[253]	validation-rmse:6.49563                                                                                          
[254]	validation-rmse:6.49537                                                                                          
[255]	validation-rmse:6.49512                                                                                          
[256]	validation-rmse:6.49488                                                                                          
[257]	validation-rmse:6.49468           

...

[478]	validation-rmse:6.57885                                                                                          
[479]	validation-rmse:6.57880                                                                                          
[480]	validation-rmse:6.57877                                                                                          
[481]	validation-rmse:6.57884                                                                                          
[482]	validation-rmse:6.57882                                                                                          
[483]	validation-rmse:6.57878                                                                                          
[484]	validation-rmse:6.57879                                                                                          
[485]	validation-rmse:6.57871                                                                                          
[486]	validation-rmse:6.57862           

[614]	validation-rmse:6.57490                                                                                          
[615]	validation-rmse:6.57494                                                                                          
[616]	validation-rmse:6.57494                                                                                          
[617]	validation-rmse:6.57489                                                                                          
[618]	validation-rmse:6.57488                                                                                          
[619]	validation-rmse:6.57489                                                                                          
[620]	validation-rmse:6.57489                                                                                          
[621]	validation-rmse:6.57492                                                                                          
[622]	validation-rmse:6.57491           

[750]	validation-rmse:6.57259                                                                                          
[751]	validation-rmse:6.57255                                                                                          
[752]	validation-rmse:6.57249                                                                                          
[753]	validation-rmse:6.57246                                                                                          
[754]	validation-rmse:6.57245                                                                                          
[755]	validation-rmse:6.57244                                                                                          
[756]	validation-rmse:6.57243                                                                                          
[757]	validation-rmse:6.57243                                                                                          
[758]	validation-rmse:6.57240           

[886]	validation-rmse:6.57079                                                                                          
[887]	validation-rmse:6.57074                                                                                          
[888]	validation-rmse:6.57075                                                                                          
[889]	validation-rmse:6.57073                                                                                          
[890]	validation-rmse:6.57064                                                                                          
[891]	validation-rmse:6.57063                                                                                          
[892]	validation-rmse:6.57059                                                                                          
[893]	validation-rmse:6.57055                                                                                          
[894]	validation-rmse:6.57055           

[20]	validation-rmse:6.50438                                                                                           
[21]	validation-rmse:6.50296                                                                                           
[22]	validation-rmse:6.50120                                                                                           
[23]	validation-rmse:6.49980                                                                                           
[24]	validation-rmse:6.49821                                                                                           
[25]	validation-rmse:6.49734                                                                                           
[26]	validation-rmse:6.49628                                                                                           
[27]	validation-rmse:6.49366                                                                                           
[28]	validation-rmse:6.49160            

### Train the best model according to the metric defined using the hyperparameters chosen by Hyperopt
- some frameworks allows autologging
https://www.mlflow.org/docs/latest/tracking.html#automatic-logging

- autolog , as the name suggests, logs different parameters, metrics without explicitly logging them, as we previously done

In [11]:
params = {
    'learning_rate':0.08993099095725796,
    'max_depth':70,
    'min_child_weight':1.0916141235072994,
    'objective':'reg:linear',
    'reg_alpha':2.207232242954968,
    'reg_lambda':0.013684231333486388
}

mlflow.xgboost.autolog()

booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid,'validation')],
            early_stopping_rounds=50
        )

2022/08/08 20:43:29 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '76223761b0ae44a5aa8080312fb0255e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[20:43:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.58018
[1]	validation-rmse:18.12822
[2]	validation-rmse:16.82001
[3]	validation-rmse:15.64426
[4]	validation-rmse:14.58765
[5]	validation-rmse:13.64205
[6]	validation-rmse:12.79593
[7]	validation-rmse:12.04179
[8]	validation-rmse:11.37229
[9]	validation-rmse:10.77601
[10]	validation-rmse:10.24789
[11]	validation-rmse:9.78225
[12]	validation-rmse:9.37035
[13]	validation-rmse:9.00525
[14]	validation-rmse:8.68744
[15]	validation-rmse:8.40715
[16]	validation-rmse:8.16061
[17]	validation-rmse:7.94432
[18]	validation-rmse:7.75699
[19]	validation-rmse:7.59475
[20]	validation-rmse:7.45171
[21]	validation-rmse:7.32848
[22]	validation-rmse:7.21909
[23]	validation-rmse:7.12608
[24]	validation-rmse:7.04245
[25]	validation-rmse:6.96972
[26]	validation-rmse:6.90569
[27]	validation-rmse:6.85001
[28]	valid

[271]	validation-rmse:6.31875
[272]	validation-rmse:6.31868
[273]	validation-rmse:6.31850
[274]	validation-rmse:6.31842
[275]	validation-rmse:6.31827
[276]	validation-rmse:6.31808
[277]	validation-rmse:6.31789
[278]	validation-rmse:6.31777
[279]	validation-rmse:6.31755
[280]	validation-rmse:6.31764
[281]	validation-rmse:6.31763
[282]	validation-rmse:6.31753
[283]	validation-rmse:6.31723
[284]	validation-rmse:6.31718
[285]	validation-rmse:6.31706
[286]	validation-rmse:6.31699
[287]	validation-rmse:6.31687
[288]	validation-rmse:6.31672
[289]	validation-rmse:6.31657
[290]	validation-rmse:6.31660
[291]	validation-rmse:6.31629
[292]	validation-rmse:6.31607
[293]	validation-rmse:6.31592
[294]	validation-rmse:6.31569
[295]	validation-rmse:6.31558
[296]	validation-rmse:6.31543
[297]	validation-rmse:6.31546
[298]	validation-rmse:6.31522
[299]	validation-rmse:6.31507
[300]	validation-rmse:6.31513
[301]	validation-rmse:6.31505
[302]	validation-rmse:6.31505
[303]	validation-rmse:6.31490
[304]	vali

[545]	validation-rmse:6.30473
[546]	validation-rmse:6.30462
[547]	validation-rmse:6.30462
[548]	validation-rmse:6.30445
[549]	validation-rmse:6.30441
[550]	validation-rmse:6.30454
[551]	validation-rmse:6.30440
[552]	validation-rmse:6.30436


Matplotlib is building the font cache; this may take a moment.
2022/08/08 20:49:03 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2022/08/08 20:49:11 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\pedro\anaconda3\envs\env-experiment-tracking\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils."


### How to log model as artifact?

In [18]:
params = {
    'learning_rate':0.08993099095725796,
    'max_depth':70,
    'min_child_weight':1.0916141235072994,
    'objective':'reg:linear',
    'reg_alpha':2.207232242954968,
    'reg_lambda':0.013684231333486388
}

mlflow.log_params(params)

booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid,'validation')],
            early_stopping_rounds=50
        )

mlflow.xgboost.log_model(booster,artifact_path='model_mlflow')

[21:13:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.58018
[1]	validation-rmse:18.12822
[2]	validation-rmse:16.82001
[3]	validation-rmse:15.64426
[4]	validation-rmse:14.58765
[5]	validation-rmse:13.64205
[6]	validation-rmse:12.79593
[7]	validation-rmse:12.04179
[8]	validation-rmse:11.37229
[9]	validation-rmse:10.77601
[10]	validation-rmse:10.24789
[11]	validation-rmse:9.78225
[12]	validation-rmse:9.37035
[13]	validation-rmse:9.00525
[14]	validation-rmse:8.68744
[15]	validation-rmse:8.40715
[16]	validation-rmse:8.16061
[17]	validation-rmse:7.94432
[18]	validation-rmse:7.75699
[19]	validation-rmse:7.59475
[20]	validation-rmse:7.45171
[21]	validation-rmse:7.32848
[22]	validation-rmse:7.21909
[23]	validation-rmse:7.12608
[24]	validation-rmse:7.04245
[25]	validation-rmse:6.96972
[26]	validation-rmse:6.90569
[27]	validation-rmse:6.85001
[28]	valid

[271]	validation-rmse:6.31875
[272]	validation-rmse:6.31868
[273]	validation-rmse:6.31850
[274]	validation-rmse:6.31842
[275]	validation-rmse:6.31827
[276]	validation-rmse:6.31808
[277]	validation-rmse:6.31789
[278]	validation-rmse:6.31777
[279]	validation-rmse:6.31755
[280]	validation-rmse:6.31764
[281]	validation-rmse:6.31763
[282]	validation-rmse:6.31753
[283]	validation-rmse:6.31723
[284]	validation-rmse:6.31718
[285]	validation-rmse:6.31706
[286]	validation-rmse:6.31699
[287]	validation-rmse:6.31687
[288]	validation-rmse:6.31672
[289]	validation-rmse:6.31657
[290]	validation-rmse:6.31660
[291]	validation-rmse:6.31629
[292]	validation-rmse:6.31607
[293]	validation-rmse:6.31592
[294]	validation-rmse:6.31569
[295]	validation-rmse:6.31558
[296]	validation-rmse:6.31543
[297]	validation-rmse:6.31546
[298]	validation-rmse:6.31522
[299]	validation-rmse:6.31507
[300]	validation-rmse:6.31513
[301]	validation-rmse:6.31505
[302]	validation-rmse:6.31505
[303]	validation-rmse:6.31490
[304]	vali

[545]	validation-rmse:6.30473
[546]	validation-rmse:6.30462
[547]	validation-rmse:6.30462
[548]	validation-rmse:6.30445
[549]	validation-rmse:6.30441
[550]	validation-rmse:6.30454
[551]	validation-rmse:6.30440


2022/08/08 21:19:25 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2022/08/08 21:19:32 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 9389075a5c634fc6ae137cbf95dce021. Failed operations: [MlflowException("Changing param values is not allowed. Params were already logged=\'[{\'key\': \'num_boost_round\', \'old_value\': None, \'new_value\': \'1000\'}, {\'key\': \'early_stopping_rounds\', \'old_value\': None, \'new_value\': \'50\'}, {\'key\': \'maximize\', \'old_value\': None, \'new_value\': \'None\'}, {\'key\': \'verbose_eval\', \'old_value\': None, \'new_value\': \'True\'}, {\'key\': \'custom_metric\', \'old_value\': None, \'new_value\': \'None\'}]\' f

ModelInfo(artifact_path='model_mlflow', flavors={'python_function': {'loader_module': 'mlflow.xgboost', 'python_version': '3.9.12', 'data': 'model.xgb', 'env': 'conda.yaml'}, 'xgboost': {'xgb_version': '1.6.1', 'data': 'model.xgb', 'model_class': 'xgboost.core.Booster', 'code': None}}, model_uri='runs:/9389075a5c634fc6ae137cbf95dce021/model_mlflow', model_uuid='59524dcf5fc045e3a95dbd362d065be7', run_id='9389075a5c634fc6ae137cbf95dce021', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-08-09 00:19:32.889257', mlflow_version='1.27.0')

### How to log model and preprocessor?

In [11]:
#disable autolog model to prevent duplicating model files
mlflow.xgboost.autolog(disable=True)

In [12]:
params = {
    'learning_rate':0.08993099095725796,
    'max_depth':70,
    'min_child_weight':1.0916141235072994,
    'objective':'reg:linear',
    'reg_alpha':2.207232242954968,
    'reg_lambda':0.013684231333486388
}

mlflow.log_params(params)

booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid,'validation')],
            early_stopping_rounds=50
        )

y_pred = booster.predict(valid)
rmse = mean_squared_error(y_val, y_pred, squared=False)
mlflow.log_metric('rmse',rmse)

with open('models/preprocessor.b','wb') as f_out:
    pickle.dump(dv,f_out)

mlflow.log_artifact(local_path='models/preprocessor.b',artifact_path='preprocessor')

mlflow.xgboost.log_model(booster,artifact_path='model_mlflow')

[22:44:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.58018
[1]	validation-rmse:18.12822
[2]	validation-rmse:16.82001
[3]	validation-rmse:15.64426
[4]	validation-rmse:14.58765
[5]	validation-rmse:13.64205
[6]	validation-rmse:12.79593
[7]	validation-rmse:12.04179
[8]	validation-rmse:11.37229
[9]	validation-rmse:10.77601
[10]	validation-rmse:10.24789
[11]	validation-rmse:9.78225
[12]	validation-rmse:9.37035
[13]	validation-rmse:9.00525
[14]	validation-rmse:8.68744
[15]	validation-rmse:8.40715
[16]	validation-rmse:8.16061
[17]	validation-rmse:7.94432
[18]	validation-rmse:7.75699
[19]	validation-rmse:7.59475
[20]	validation-rmse:7.45171
[21]	validation-rmse:7.32848
[22]	validation-rmse:7.21909
[23]	validation-rmse:7.12608
[24]	validation-rmse:7.04245
[25]	validation-rmse:6.96972
[26]	validation-rmse:6.90569
[27]	validation-rmse:6.85001
[28]	valid

[271]	validation-rmse:6.31875
[272]	validation-rmse:6.31868
[273]	validation-rmse:6.31850
[274]	validation-rmse:6.31842
[275]	validation-rmse:6.31827
[276]	validation-rmse:6.31808
[277]	validation-rmse:6.31789
[278]	validation-rmse:6.31777
[279]	validation-rmse:6.31755
[280]	validation-rmse:6.31764
[281]	validation-rmse:6.31763
[282]	validation-rmse:6.31753
[283]	validation-rmse:6.31723
[284]	validation-rmse:6.31718
[285]	validation-rmse:6.31706
[286]	validation-rmse:6.31699
[287]	validation-rmse:6.31687
[288]	validation-rmse:6.31672
[289]	validation-rmse:6.31657
[290]	validation-rmse:6.31660
[291]	validation-rmse:6.31629
[292]	validation-rmse:6.31607
[293]	validation-rmse:6.31592
[294]	validation-rmse:6.31569
[295]	validation-rmse:6.31558
[296]	validation-rmse:6.31543
[297]	validation-rmse:6.31546
[298]	validation-rmse:6.31522
[299]	validation-rmse:6.31507
[300]	validation-rmse:6.31513
[301]	validation-rmse:6.31505
[302]	validation-rmse:6.31505
[303]	validation-rmse:6.31490
[304]	vali

[545]	validation-rmse:6.30473
[546]	validation-rmse:6.30462
[547]	validation-rmse:6.30462
[548]	validation-rmse:6.30445
[549]	validation-rmse:6.30441
[550]	validation-rmse:6.30454
[551]	validation-rmse:6.30440
[552]	validation-rmse:6.30436


C:\Users\pedro\anaconda3\envs\env-experiment-tracking\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


ModelInfo(artifact_path='model_mlflow', flavors={'python_function': {'loader_module': 'mlflow.xgboost', 'python_version': '3.9.12', 'data': 'model.xgb', 'env': 'conda.yaml'}, 'xgboost': {'xgb_version': '1.6.1', 'data': 'model.xgb', 'model_class': 'xgboost.core.Booster', 'code': None}}, model_uri='runs:/8a44f2719fe5458db1e7d926838650fa/model_mlflow', model_uuid='10a15c18da7240c0a9e1c1ee32832930', run_id='8a44f2719fe5458db1e7d926838650fa', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-08-14 01:48:23.289775', mlflow_version='1.27.0')

### Make predictions using logged model

In [13]:
logged_model = 'runs:/8a44f2719fe5458db1e7d926838650fa/model_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

[22:51:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [14]:
#model loaded as pyfunc
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model_mlflow
  flavor: mlflow.xgboost
  run_id: 8a44f2719fe5458db1e7d926838650fa

In [15]:
#model loaded xgboost object
xgboost_model = mlflow.xgboost.load_model(logged_model)

[22:52:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [16]:
#make predictions
y_pred = xgboost_model.predict(valid)
y_pred[:10]

array([14.564945 ,  6.6839285, 15.387255 , 24.285564 ,  9.326031 ,
       17.26144  , 10.963047 ,  8.0557785,  9.274415 , 20.114908 ],
      dtype=float32)